In [1]:
from dataclasses import dataclass, field
from typing import Optional
import huggingface_hub
import functools as ft
import torch
import pandas as pd
import torch
from accelerate import Accelerator
from datasets import load_dataset
from peft import LoraConfig
from tqdm import tqdm
from transformers import Adafactor, AutoTokenizer, HfArgumentParser, pipeline, AutoConfig, GPTNeoXForCausalLM, AutoModelForCausalLM
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer, set_seed
from trl.core import LengthSampler
from transformers import pipeline, TextGenerationPipeline, AutoConfig, AutoTokenizer, AutoModelForCausalLM, AutoModel, AutoModelForSequenceClassification, GPTNeoXForCausalLM, LlamaForSequenceClassification
from redditqa.dataset import load_reddit_dataset
from transformers.utils.hub import convert_file_size_to_int, get_checkpoint_shard_files

/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-07-30 15:51:47.235091: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-30 15:51:47.738468: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
model_checkpoint = '/scratch1/jhoff/checkpoints/reward_llama-2-7b-chat-hf/checkpoint-3000'

In [7]:
adapter_weights = torch.load(f'{model_checkpoint}/adapter_model.bin', map_location='cpu')
print('\n'.join(list(adapter_weights.keys())))

base_model.model.model.layers.0.self_attn.q_proj.lora_A.weight
base_model.model.model.layers.0.self_attn.q_proj.lora_B.weight
base_model.model.model.layers.0.self_attn.v_proj.lora_A.weight
base_model.model.model.layers.0.self_attn.v_proj.lora_B.weight
base_model.model.model.layers.1.self_attn.q_proj.lora_A.weight
base_model.model.model.layers.1.self_attn.q_proj.lora_B.weight
base_model.model.model.layers.1.self_attn.v_proj.lora_A.weight
base_model.model.model.layers.1.self_attn.v_proj.lora_B.weight
base_model.model.model.layers.2.self_attn.q_proj.lora_A.weight
base_model.model.model.layers.2.self_attn.q_proj.lora_B.weight
base_model.model.model.layers.2.self_attn.v_proj.lora_A.weight
base_model.model.model.layers.2.self_attn.v_proj.lora_B.weight
base_model.model.model.layers.3.self_attn.q_proj.lora_A.weight
base_model.model.model.layers.3.self_attn.q_proj.lora_B.weight
base_model.model.model.layers.3.self_attn.v_proj.lora_A.weight
base_model.model.model.layers.3.self_attn.v_proj.lora_B

In [3]:
shard_files = get_checkpoint_shard_files(model_checkpoint, index_filename=f'{model_checkpoint}/pytorch_model.bin.index.json')

for shard_file in shard_files[0]:
    
    manually_loaded_weights = torch.load(shard_files[0][2], map_location='cpu')

    print('\n'.join(list(manually_loaded_weights.keys())))

ValueError: Can't find a checkpoint index (/scratch1/jhoff/checkpoints/reward_llama-2-7b-chat-hf/checkpoint-3000/pytorch_model.bin.index.json) in /scratch1/jhoff/checkpoints/reward_llama-2-7b-chat-hf/checkpoint-3000.

In [10]:
adapter_weights['base_model.model.score.weight']

tensor([[ 0.0381,  0.0195,  0.0079,  ..., -0.0181, -0.0104, -0.0177]],
       dtype=torch.bfloat16)